In [1]:
import joblib
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

In [74]:
min_coverage = 10

In [158]:
data[(data["BASE_GT"] != "REF") & (data["svtype"] == 'INS')].groupby('SAMPLE').size().describe()

count       33.000000
mean     15445.424242
std       1070.207487
min      14439.000000
25%      14689.000000
50%      14773.000000
75%      16852.000000
max      17107.000000
dtype: float64

In [134]:
len(data[(data["NumNeighbors"] != 0) & (data["svtype"] == "INS")]) / len(data)

0.5624031361108579

In [150]:
data = joblib.load("../stats/genotyping.biograph.grch38.jl")

data["TRF"] = data["TRF"].where(~data["TRF"].isna(), False)
data["state"] = data["BASE_GT"] == data["COMP_GT"]
data["DP"] = data["AD_ref"] + data["AD_alt"]
#keep = (data["DP"] >= min_coverage) & (data["FT"]) & (data["COMP_GT"].isin(["REF", "HET", "HOM"]))
keep = (data["DP"] >= min_coverage) & (data["COMP_GT"].isin(["REF", "HET", "HOM"]))
# Do FT filtering on Paragraph

In [153]:
# Per-sample TP percent
data[data["BASE_GT"] != "REF"].groupby(['SAMPLE'])['state'].mean().describe()

count    33.000000
mean      0.510089
std       0.016852
min       0.457630
25%       0.499715
50%       0.508751
75%       0.525304
max       0.536798
Name: state, dtype: float64

In [154]:
data[(data["BASE_GT"] != "REF") & data['state']].groupby(['SAMPLE']).size().describe()

count       33.000000
mean     12867.545455
std       1351.585946
min      10871.000000
25%      11856.000000
50%      12204.000000
75%      14511.000000
max      14997.000000
dtype: float64

In [155]:
#TN-Rate
data[data["BASE_GT"] == "REF"].groupby(['SAMPLE'])['state'].mean().describe()

count    33.000000
mean      0.913153
std       0.005618
min       0.887385
25%       0.912170
50%       0.913124
75%       0.915936
max       0.925816
Name: state, dtype: float64

In [156]:
data[(data["BASE_GT"] == "REF") & ~data['state']].groupby(['SAMPLE']).size().describe()

count       33.000000
mean     14003.333333
std        839.296940
min      12071.000000
25%      13701.000000
50%      13908.000000
75%      14193.000000
max      17899.000000
dtype: float64

In [136]:
data['BASE_GT'].value_counts() / len(data)

REF    0.864975
HET    0.101109
HOM    0.033917
Name: BASE_GT, dtype: float64

In [137]:
print("Percent of sites filtered %.1f%%" % ( (1 - keep.sum() / len(data)) * 100))

Percent of sites filtered 40.6%


In [138]:
rows = []

all_filter = ('TOT', data['svtype'].isin(["DEL", "INS"]))
del_filter = ('DEL', data['svtype'] == 'DEL')
ins_filter = ('INS', data['svtype'] == 'INS')

all_neigh_filter = ('.', data["NumNeighbors"] != -23)
non_neigh_filter = ("None", data["NumNeighbors"] == 0)
has_neigh_filter = ("Any", data["NumNeighbors"] != 0)

for label1, sv_filt in [all_filter, del_filter, ins_filter]:
    for label2, neigh_filt in [all_neigh_filter, non_neigh_filter, has_neigh_filter]:
        view = data[keep & sv_filt & neigh_filt]
        sv_percent = len(view) / len(data[keep])
        overall_accuracy = view["state"].mean()
        ref_accuracy = view[view["BASE_GT"] == 'REF']["state"].mean()
        alt_accuracy = view[view["BASE_GT"] != 'REF']['state'].mean()
        rows.append([sv_percent, label1, label2, overall_accuracy, ref_accuracy, alt_accuracy])

summary = pd.DataFrame(rows, columns=["pct of sites", "svtype", "NumNeighbors", "Overall", "Reference", "Alternate"])
summary
# Variables are SVTYPE [DEL, "INS"], NumNeighbors ==0 / !=0, 
# overall accuracy , ref accuracy, alt accuracy, 
# false presence % (% of sites genotyped as being present, but aren't)
# flase absence (% of sites genotyped as being REF, but should be present)
# Need to add the 'percent of SVs that fall into each BIN'. This helps reestablith the scale

,pct of sites,svtype,NumNeighbors,Overall,Reference,Alternate
0,1.000000,TOT,.,0.598828,0.576825,0.723846
1,0.297040,TOT,None,0.931706,0.940530,0.897868
2,0.702960,TOT,Any,0.458167,0.437432,0.602659
3,0.372856,DEL,.,0.729165,0.719232,0.779317
4,0.137429,DEL,None,0.954970,0.964104,0.922350
5,0.235427,DEL,Any,0.597352,0.590266,0.643106
6,0.627144,INS,.,0.521338,0.494617,0.685004
7,0.159611,INS,None,0.911676,0.920793,0.874405
8,0.467533,INS,Any,0.388081,0.361604,0.580117


In [139]:
summary.to_csv('bg.txt', sep='\t', index=False)

In [86]:
# How many variants are there on reference?
# How many variants are filtered
# What's the filtered accuracy
# What's the filtered accuracy by type
# What's the filered accuracy by NumNeighbors
# Any difference by REMAP - sorta - novel are the highest and tandem is lowest.

,AF,MAF,svtype,szbin,REMAP,NumCollapsed,TRF,TRFperiod,TRFcopies,NumNeighbors,...,ASMCOVDN,SAMPLE,BASE_GT,COMP_GT,AD_ref,AD_alt,GQ,FT,state,DP
key,,,,,,,,,,,,,,,,,,,,,
chr10:100092182-100092183.CCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCT,0.027778,0.027778,INS,"[200,300)",interspersed,1.0,True,6.0,60.299999,2,...,59,HG00096,REF,REF,34,4,369.0,True,True,38
chr10:100092182-100092183.CTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCT,0.013889,0.013889,INS,"[300,400)",interspersed,NaN,True,6.0,77.300003,2,...,59,HG00096,REF,REF,34,4,369.0,True,True,38
chr10:100092192-100092193.TCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTCCCTCTC,0.013889,0.013889,INS,"[200,300)",interspersed,NaN,True,6.0,56.299999,2,...,59,HG00096,REF,REF,36,4,393.0,True,True,40
chr10:100093489-100093490.TGGGGGGGGTCAGCCCCCCTGCCCGGCCAGCCGCCCCGTCCGGGAGGTGAGGGGCGCCTCTGCCCGGCCGCCCCTACTGGGAAGTGAGGAGCCCCTCTGCCCGGCCAGCTGCCCCGTCCGCGAGGGAGG,0.027778,0.027778,INS,"[100,200)",interspersed,NaN,True,127.0,2.700000,6,...,72,HG00096,REF,HET,30,47,30.0,False,False,77
chr10:100093500-100093501.GGTCAGCCCCCCTGCCCGGCCAGCCGCCCCGTCCGGGAGGGAGGTGGGGGGGGGTCAGCCCCCCTGCCCGGCCAGCCGCCCCGTCCGGGAGGTGAGGGGCGCCTCTGCCCGGCCGCCCCTACTGGGAAGTGAGGAGCCCCTCTGCCCGGCCAGCTGCCCCGTCCGCGAGGGAGGTGGGGGGGGTCAGCCCCCCGCCCGGCCAGCTGCCCCATCCGGGAGGGAGGTGGGGGGGGTCAGC,0.013889,0.013889,INS,"[200,300)",interspersed,NaN,True,127.0,7.700000,6,...,72,HG00096,REF,HET,25,20,195.0,True,False,45


SAMPLE
HG00096    0.653538
HG00171    0.657324
HG00512    0.646847
HG00513    0.657125
HG00514    0.649975
HG00731    0.648847
HG00732    0.645362
HG00733    0.667391
HG00864    0.646684
HG01114    0.653047
HG01505    0.641361
HG01596    0.645268
HG02011    0.662555
HG02492    0.649420
HG02587    0.651833
HG02818    0.665492
HG03009    0.642054
HG03065    0.656387
HG03125    0.665671
HG03371    0.656513
HG03486    0.664773
HG03683    0.654681
HG03732    0.654716
NA12329    0.650948
NA18534    0.642084
NA18939    0.643001
NA19238    0.660916
NA19239    0.649631
NA19240    0.654202
NA19650    0.661382
NA19983    0.654113
NA20509    0.654208
NA20847    0.650068
Name: state, dtype: float64

In [87]:
# On average, samples have genotyping accuracy of

In [98]:
data[keep].groupby(['REMAP'])['state'].mean()

REMAP
interspersed    0.677802
novel           0.781912
partial         0.686782
tandem          0.449701
Name: state, dtype: float64

In [89]:
# how many variants were genotyped

In [90]:
data[keep].groupby(['SAMPLE']).size().describe()

count        33.000000
mean     110827.272727
std        2090.137799
min      107738.000000
25%      109537.000000
50%      110580.000000
75%      111754.000000
max      119255.000000
dtype: float64

In [91]:
# However, N% of variants in the graph-reference are ref HOM
(data.groupby(['SAMPLE', 'BASE_GT']).size() / 186473).loc[:, 'REF'].describe()

count    33.000000
mean      0.864975
std       0.010327
min       0.848938
25%       0.851298
50%       0.871595
75%       0.872738
max       0.874025
dtype: float64

In [92]:
# Therefore, looking at the genotyping accuracy of only present sites

In [93]:
data[(keep) & (data["BASE_GT"] != "REF")].groupby(['SAMPLE'])['state'].mean().describe()

count    33.000000
mean      0.723652
std       0.007083
min       0.711968
25%       0.718237
50%       0.722258
75%       0.727606
max       0.739166
Name: state, dtype: float64

In [94]:
# There is a difference by SVTYPE
data[(keep) & (data["BASE_GT"] != "REF")].groupby(['SAMPLE', 'svtype'])['state'].mean().unstack()[["DEL", "INS"]].describe()

svtype,DEL,INS
count,33.000000,33.000000
mean,0.778690,0.685117
std,0.009154,0.008383
min,0.764849,0.663459
25%,0.772061,0.681329
50%,0.777367,0.684913
75%,0.785029,0.689548
max,0.797981,0.705369


In [95]:
# But the bit difference is by if there's another call nearby (within numneigh)
data[(keep) & (data["BASE_GT"] != "REF") & (data["NumNeighbors"] == 0)].groupby(['SAMPLE', 'svtype'])['state'].mean().unstack()[["DEL", "INS"]].describe()

svtype,DEL,INS
count,33.000000,33.000000
mean,0.922017,0.874445
std,0.005894,0.012107
min,0.908555,0.834865
25%,0.917862,0.872152
50%,0.922674,0.876270
75%,0.925717,0.881287
max,0.933226,0.894393


In [96]:
data[(keep) & (data["BASE_GT"] != "REF") 
     & (data["NumNeighbors"] != 0)].groupby(['SAMPLE', 'svtype'])['state'].mean().unstack()[["DEL", "INS"]].describe()



svtype,DEL,INS
count,33.000000,33.000000
mean,0.642380,0.580434
std,0.014098,0.010267
min,0.616840,0.559496
25%,0.632084,0.575622
50%,0.643318,0.580213
75%,0.650393,0.584881
max,0.678015,0.609581


In [97]:
# And there's a difference in accuracy by the BASE GT type
data[keep].groupby(['SAMPLE', 'BASE_GT'])['state'].mean().unstack().describe()

BASE_GT,HET,HOM,REF
count,33.000000,33.000000,33.000000
mean,0.666418,0.864326,0.576681
std,0.014434,0.007461,0.015343
min,0.639163,0.851233,0.544987
25%,0.657736,0.857236,0.561375
50%,0.663963,0.865695,0.583242
75%,0.678350,0.870232,0.587740
max,0.693615,0.876554,0.599044


In [46]:
# Labeled genotype concordance. (percent of BASE GTs with correct label) (akin to recall)

Supplementary will be the opposite - predicted GT percent. Which will show 'if the genotyper says it's REF, it is ref N % of the time
akin to specificity
I'm trying to say you can't use genotypes to validate SVs. Once you give the genotypers a prior, that has a stong effect on the outcome. This is particularaly true for Paragraph. So 

- N vars per sample multipled out - how many sites we're genotyping
- We want to see how many of the sites are filtered out
- We want to see the overall accuracy of the remaining sites
- We want to see the accuracy of the remaining sites by Genotype
- We want to see the accuracy of the remaining sites by NumNeigh

At the end, I want to be able to say that 
1. Short reads can genotype ~X% of SVs with a bias against those in 'dense' regions
2. BioGraph is probably better. 
3. When genotyping (especially) with short-reads, we have to be careful about how we check the accuracy of the calller. Just doing a simple 'presence' check (and not concordance) on a single sample can greatly over-estimate the caller's performance. If you guess 'present' too frequently  you won't notice how many False you're getting because most sites are REF.
Damn this is going to be hard to summarize.
